In [1]:
!pip install git+https://github.com/openai/whisper.git -q

You should consider upgrading via the '/Users/bpakra200/tubegpt/gptenv/bin/python3 -m pip install --upgrade pip' command.


In [33]:
!pip install --upgrade --force-reinstall "git+https://github.com/ytdl-org/youtube-dl.git"

  Cloning https://github.com/ytdl-org/youtube-dl.git to /private/var/folders/2h/dwgxsn2s7k76kf3hl_8gpkz40000gp/T/pip-req-build-mc0rebr6
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /private/var/folders/2h/dwgxsn2s7k76kf3hl_8gpkz40000gp/T/pip-req-build-mc0rebr6
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit 211cbfd5d46025a8e4d8f9f3d424aaada4698974
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for youtube-dl, since package 'wheel' is not installed.
  Attempting uninstall: youtube-dl
    Found existing installation: youtube-dl 2021.12.17
    Uninstalling youtube-dl-2021.12.17:
      Successfully uninstalled youtube-dl-2021.12.17
  Running setup.py install for youtube-dl ... done
You should consider upgrading via the '/Users/bpakra200/tubegpt/gptenv/bin/python3 -m pip install --upgrade pip' command.


In [27]:
!pip install pydub

You should consider upgrading via the '/Users/bpakra200/tubegpt/gptenv/bin/python3 -m pip install --upgrade pip' command.


In [1]:

import whisper

In [2]:

model = whisper.load_model('base')

In [3]:
import youtube_dl # client to download from many multimedia portals
import glob # directory operations
import os # interface to os-provided info on files
import sys # interface to command line
from pydub import AudioSegment # only audio operations

def newest_mp3_filename():
    # lists all mp3s in local directory
    list_of_mp3s = glob.glob('./*.mp3')
    # returns mp3 with highest timestamp value
    return max(list_of_mp3s, key = os.path.getctime)

def get_video_time_in_ms(video_timestamp):
    vt_split = video_timestamp.split(":")
    if (len(vt_split) == 3): # if in HH:MM:SS format
        hours = int(vt_split[0]) * 60 * 60 * 1000
        minutes = int(vt_split[1]) * 60 * 1000
        seconds = int(vt_split[2]) * 1000
    else: # MM:SS format
        hours = 0
        minutes = int(vt_split[0]) * 60 * 1000
        seconds = int(vt_split[1]) * 1000
    # time point in miliseconds
    return hours + minutes + seconds

def get_trimmed(mp3_filename, initial, final = ""):
    if (not mp3_filename):
        # raise an error to immediately halt program execution
        raise Exception("No MP3 found in local directory.")
    # reads mp3 as a PyDub object
    sound = AudioSegment.from_mp3(mp3_filename)
    t0 = get_video_time_in_ms(initial)
    print("Beginning trimming process for file ", mp3_filename, ".\n")
    print("Starting from ", initial, "...")
    if (len(final) > 0):
        print("...up to ", final, ".\n")
        t1 = get_video_time_in_ms(final)
        return sound[t0:t1] # t0 up to t1
    return sound[t0:] # t0 up to the end



# downloads yt_url to the same directory from which the script runs
def download_audio(yt_url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([yt_url])

def main(yt_url,initial,final):
    download_audio(yt_url)
    filename = newest_mp3_filename()
    trimmed_file = get_trimmed(filename, initial, final)
    trimmed_filename = "".join([filename.split(".mp3")[0], "- TRIM.mp3"])
    print("Process concluded successfully. Saving trimmed file as ", trimmed_filename)
    # saves file with newer filename
    trimmed_file.export(trimmed_filename, format="mp3")

# example usage:
# python ytauddown.py https://www.youtube.com/watch?v=8OAPLk20epo 9:51 14:04


/Users/bpakra200/tubegpt/gptenv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [11]:
main('https://www.youtube.com/watch?v=6RClsEZuXtE' , '0:01' ,'1:51')

[youtube] 6RClsEZuXtE: Downloading webpage
[youtube] 6RClsEZuXtE: Downloading MPD manifest
[dashsegments] Total fragments: 1
[download] Destination: FOOD DELIVERY. ORDERING FAST FOOD.-6RClsEZuXtE.m4a
[download] 100% of 1.93MiB in 00:00.95MiB/s ETA 00:002
[ffmpeg] Correcting container in "FOOD DELIVERY. ORDERING FAST FOOD.-6RClsEZuXtE.m4a"
[ffmpeg] Destination: FOOD DELIVERY. ORDERING FAST FOOD.-6RClsEZuXtE.mp3
Deleting original file FOOD DELIVERY. ORDERING FAST FOOD.-6RClsEZuXtE.m4a (pass -k to keep)
Beginning trimming process for file  ./FOOD DELIVERY. ORDERING FAST FOOD.-6RClsEZuXtE.mp3 .

Starting from  0:01 ...
...up to  1:51 .

Process concluded successfully. Saving trimmed file as  ./FOOD DELIVERY. ORDERING FAST FOOD.-6RClsEZuXtE- TRIM.mp3


In [12]:

import datetime

# save a timestamp before transcription
t1 = datetime.datetime.now()
print(f"started at {t1}")

# do the transcription
output = model.transcribe("./FOOD DELIVERY. ORDERING FAST FOOD.-6RClsEZuXtE- TRIM.mp3")

# show time elapsed after transcription is complete.
t2 = datetime.datetime.now()
print(f"ended at {t2}")
print(f"time elapsed: {t2 - t1}")

started at 2023-05-01 21:57:24.043952


/Users/bpakra200/tubegpt/gptenv/lib/python3.9/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


ended at 2023-05-01 21:57:32.771477
time elapsed: 0:00:08.727525


In [13]:
output['text']

" Good morning, I'm Burger Delivery. How can I help you? Hi, I'd like to order food please. Pick up or delivery? Delivery. Okay, what would you like to order? Could I have one combo number six and four salads and three pizzas please? What dressing would you like? Rinse dressing and could you send some extra chili sauce please? Sure, would you like any drinks? We have a big assortment of soft drinks. No, thanks. Okay, anything else? No, thanks. That's all. Thank you, what's your address please? 25 green street apartment five. My buzzer code is 147. Okay, thank you. Can I have your name and phone number as well? My name is Dasha. My mobile is 555-7364. Thank you Dasha. How are you paying? What ways of payment are available? Typical payment methods include cash, checks, credit or debit cards, money orders, bank transfers and online payment services such as PayPal. What type of payment do you prefer? In cash. How much will it be? 50 dollars. You can pay your order cash to directly courier 

In [2]:
import torch
import torchvision.transforms as transforms
import cv2
from PIL import Image
import pafy
from transformers import BlipProcessor, BlipForConditionalGeneration
import youtube_dl

model_id = "Salesforce/blip-image-captioning-base"

In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.agents import tool
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
import textwrap
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.tools import Tool
import langchain
import json
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain.schema import Document
from typing import Iterator, List, Literal, Optional, Sequence, Union
from langchain.document_loaders.blob_loaders import FileSystemBlobLoader
import os
import openai